# PDF Parsing Libraries

In [20]:
from thefuzz import fuzz
from pathlib import Path
from pypdf import PdfReader
from pdfminer.high_level import extract_text
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
from pdfminer.pdftypes import resolve1
import pymupdf
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [8]:
PDF_FILES_PATH = Path.cwd() / '../../data/raw/test pdfs'
GROUND_TRUTHS = Path.cwd() /  Path.cwd() / '../../data/raw/ground truth'
LIBRARY_OUTPUT = Path.cwd() / '../../data/text extractions'
MANUAL_PDF = Path.cwd() / '../../data/raw/ground truth'

### Functions Testing

In [3]:
def pypdf_txt_extraction(file_path: Path) -> dict[str, str]:
    """
    Helper function to extract text from a pdf using pypdf page-wise.

    Args:
        file_path: Path of the pdf file

    Returns:
        dict: key: page number, value: page text
    """
    pdf_dict = {}
    file = PdfReader(file_path)
    time_list = []
    for page_num in range(len(file.pages)):
        key = f'{str(file_path).split('/')[-1]} ~ {page_num}'
        start = time.time()
        extracted_txt = file.pages[page_num].extract_text()
        end = time.time()
        pdf_dict[key] = extracted_txt
        time_list.append(end-start)
    return pdf_dict, time_list

In [4]:
def pymupdf_txt_extraction(file_path: Path) -> dict[str, str]:
    """
    Helper function to extract text from a pdf using pymupdf page-wise.

    Args:
        file_path: Path of the pdf file

    Returns:
        dict: key: page number, value: page text
    """
    file = pymupdf.open(file_path)
    pdf_dict = {}
    time_list = []
    for page_num in range(file.page_count):
        start = time.time()
        text = file[page_num].get_text()
        end = time.time()
        pdf_dict[f'{str(file_path).split('/')[-1]} ~ {page_num}'] = text
        time_list.append(end-start)
    return pdf_dict, time_list

In [5]:
def pdfminersix_txt_extraction(file_path: Path) -> dict[str, str]:
    """
    Helper function to extract text from a pdf using pdfminer.six page-wise.

    Args:
        file_path: Path of the pdf file

    Returns:
        dict: key: page number, value: page text
    """
    pdf_dict = {}
    time_list = []
    with open(file_path, 'rb') as f:
        parser = PDFParser(f)
        doc = PDFDocument(parser)
        parser.set_document(doc)
        pages = resolve1(doc.catalog['Pages'])
        pages_count = pages.get('Count', 0)
    for page_num in range(pages_count):
        start = time.time()
        text = extract_text(file_path, page_numbers=[page_num])
        end = time.time()
        pdf_dict[f'{str(file_path).split('/')[-1]} ~ {page_num}'] = text
        time_list.append(end-start)
    return pdf_dict, time_list

In [6]:
pypdf_output = pypdf_txt_extraction(PDF_FILES_PATH / 'chess_pdf.pdf')
pymupdf_output = pymupdf_txt_extraction(PDF_FILES_PATH / 'chess_pdf.pdf')
pdfminer_output = pdfminersix_txt_extraction(PDF_FILES_PATH / 'chess_pdf.pdf')

In [ ]:
def levenshtein_distance(ground_truth: Path, extracted):
    # Read the contents of both files
    with open(ground_truth) as file:
        ground_content = file.read()
    # Compute and return the Levenshtein distance
    return Levenshtein.distance(ground_content, extracted)

In [ ]:
levenshtein_distance(ground_truth / 'chess_pdf.txt', )

In [7]:
pdf_df = pd.DataFrame({'PDF_ID': list(pypdf_output[0].keys())})
pdf_df['pypdf_txt'] = list((pypdf_output[0]).values())
pdf_df['pymupdf_txt'] = list((pymupdf_output[0]).values())
pdf_df['pdfminersix_text'] = list((pdfminer_output[0]).values())
pdf_df['pypdf_time'] = list(pypdf_output[1])
pdf_df['pymupdf_time'] = list(pymupdf_output[1])
pdf_df['pdfminersix_time'] = list(pdfminer_output[1])
pdf_df

,PDF_ID,pypdf_txt,pymupdf_txt,pdfminersix_text,pypdf_time,pymupdf_time,pdfminersix_time
0,chess_pdf.pdf ~ 0,Author: Prateek \n \n 1 \n \n \n Chec...,Author: Prateek \n \n \n \n1\n \n \n Checkmat...,Author: Prateek \n\n Checkmate Through T...,0.061482,0.049782,0.043715
1,chess_pdf.pdf ~ 1,"Author: Prateek \n \n 2 \n \nChess, the gam...","Author: Prateek \n \n \n \n2\n \nChess, the ga...","Author: Prateek \n\nChess, the game of kings a...",0.010423,0.004764,0.018143
2,chess_pdf.pdf ~ 2,Author: Prateek \n \n 3 \n \nThe Birth of C...,Author: Prateek \n \n \n \n3\n \nThe Birth of ...,Author: Prateek \n\nThe Birth of Competitive C...,0.008634,0.002915,0.018321


In [18]:
print(pdf_df['pypdf_time'].values.mean().round(5))
print(pdf_df['pymupdf_time'].values.mean().round(5))
print(pdf_df['pdfminersix_time'].values.mean().round(5))

0.02685
0.01915
0.02673


In [19]:
pdf_df['pypdf_time'].plot()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

---